In [618]:
import numpy as np
from sympy import symbols, And, Not, simplify, solve, reduce_inequalities
%run constants.ipynb
%run li_handler.ipynb

In [2]:
def no_loser(a,b,c):
    return [simplify(i) for i in [a>0, b>0, c>0]]

In [3]:
def parentStates(a,b,c, r=V):
    return [s for s in 
            [(2*a,b-a,c), (2*a,b,c-a),
            (a-b,2*b,c), (a,2*b,c-b),
            (a-c,b,2*c), (a,b-c,2*c)] 
            if feasible(*no_loser(*s), r=r)]

In [4]:
@memoized
def coef(n):
    return 1/(6**n)

In [5]:
def concat(ls):
    ''' Flatten 2D list. '''
    return [j for l in ls for j in l]

In [8]:
import csv
from itertools import islice

def read(filename, min_n=1, max_n=5):
    hs = {}
    with open(filename, 'r') as fp:
        reader = islice(csv.reader(fp), min_n-1, max_n)
        for row in reader:
            n, c, rs = row
            # parse entries
            hs[int(n)] = (int(c), sympify(rs))
    return hs

In [7]:
def h_mult(n,a,b,c):
    return non_redundant(*no_loser(a,b,c), r=V)

def mult_inds(inds, prereqs, r=V):
    '''
    prereqs * sum(inds), keeping only non-redundant indicators in `prereqs` given restriction `r`.
    '''
    return [i + [p for p in non_redundant(*prereqs, r=V+i)] for i in inds]

@memoized
def _h(n,a,b,c):
    ''' Returns the additive indicators of h_n(a,b,c), as a 2D list. '''
    # constant coef = (1/6)^n
    # so we only keep track of the indicators to be summed up
    if n == 1:
        return mult_inds([[simplify(a<=b)], [simplify(a<=c)]], h_mult(1,a,b,c))
    # list of inequalities representing the region Rn
    return mult_inds(concat([_h(n-1, *s) for s in parentStates(a,b,c)]), 
                     h_mult(n,a,b,c))

@memoized
def h(n,a,b,c, regenerate):
    if regenerate:
        return reduce(_h(n,a,b,c), r=V)
    else:
        if (a,b,c) == (y,x,z):
            return read(H_YXZ_CACHE, max_n=n)[n]
        else:
            return read(H_XYZ_CACHE, max_n=n)[n]

# Use xyz

In [ ]:
import gurobipy as gp
from gurobipy import GRB
from itertools import repeat

In [700]:
n = 3
H_XYZ = read(H_XYZ_CACHE, min_n=1, max_n=n)
H_YXZ = read(H_YXZ_CACHE, min_n=1, max_n=n)
all_pos_inds = [H_XYZ[i][1] for i in range(1,n+1)]
all_neg_inds = [H_YXZ[i][1] for i in range(1,n+1)]
n_pos = len(concat(concat(all_pos_inds)))
n_neg = len(concat(concat(all_neg_inds)))
pos_coefs = [coef(i) for i in range(1,n+1)]
neg_coefs = [-c for c in pos_coefs]
pos_cs = concat([repeat(j, k) for j,k in zip(pos_coefs, [len(concat(ps)) for ps in all_pos_inds])])
neg_cs = concat([repeat(j, k) for j,k in zip(neg_coefs, [len(concat(ns)) for ns in all_neg_inds])])
const = sum([coef(i)*(H_XYZ[i][0] - H_YXZ[i][0]) for i in range(1,n+1)])

In [687]:
model = gp.Model()
# model.Params.LazyConstraints = 1
model.Params.OutputFlag = 1
model.Params.LogToConsole = 1
# model.Params.MIPFocus = 1
# model.Params.Heuristics = 1
# model.Params.CliqueCuts = 1
# model.Params.FeasibilityTol = 1e-9
# model.Params.IntFeasTol = 1e-9
# model.Params.MarkowitzTol = 1e-4
# model.Params.NodeLimit = 10000
pos_inds = model.addVars(n_pos, vtype=GRB.BINARY, name='pos_inds')
neg_inds = model.addVars(n_neg, vtype=GRB.BINARY, name='neg_inds')
gx = model.addVar(lb=0, vtype='I', name='x')
gy = model.addVar(lb=1, vtype='I', name='y')
gz = model.addVar(lb=2, vtype='I', name='z')

In [688]:
obj = gp.quicksum([pos_cs[i]*pos_inds[i] for i in range(n_pos)]) + \
        gp.quicksum([neg_cs[i]*neg_inds[i] for i in range(n_neg)]) + const

In [689]:
model.setObjective(obj, GRB.MINIMIZE)
# model.addConstr(obj <= thresh(n))
model.addConstr(gx <= gy - 1)
model.addConstr(gy <= gz - 1)

<gurobi.Constr *Awaiting Model Update*>

In [690]:
all_pos_inds = concat([H_XYZ[i][1] for i in range(1,n+1)])
all_neg_inds = concat([H_YXZ[i][1] for i in range(1,n+1)])
all_neg_inds

[[3*y <= z],
 [y <= 3*x],
 [y <= 2*x],
 [x <= -y + z],
 [7*y <= z],
 [2*y <= 3*x],
 [3*y - z <= x],
 [x <= -3*y + z],
 [3*y - z <= x, z < 3*y],
 [5*y <= 3*z, z < 3*y],
 [3*y <= 5*x],
 [2*y - z <= 2*x, y < 3*x],
 [y <= 2*x],
 [3*y - z <= 3*x],
 [y <= 7*x, 3*x < y],
 [3*x < y],
 [y <= 5*x, 2*x < z],
 [x <= -y + z],
 [y + z <= 3*x],
 [z < 2*x],
 [x <= -2*y + z, y < 2*x],
 [x <= -3*y + z],
 [y <= 6*x, 2*x < y],
 [2*x < y],
 [y <= 4*x, 3*x < z],
 [3*x <= -y + z],
 [-y + z < x],
 [3*x <= -y + 3*z, -y + z < x],
 [y + z <= 3*x],
 [2*x <= -y + 2*z, z < 3*x]]

In [691]:
from sympy import symbols, Add
from gurobipy import Model, LinExpr

# Define SymPy variables
x,y,z = symbols('x y z')

# Mapping between SymPy and Gurobi variables
sympy_to_gurobi = {x: gx, y: gy, z: gz}

def to_gp_linexpr(sympy_expr, var_map=sympy_to_gurobi):
    gurobi_expr = LinExpr()

    if isinstance(sympy_expr, Add):
        for term in sympy_expr.args:
            gurobi_expr.add(sympy_to_gurobi_linexpr(term, var_map))
    elif sympy_expr.is_Mul:
        coeff = 1
        var = None
        for factor in sympy_expr.args:
            if factor.is_number:
                coeff *= float(factor)
            elif factor in var_map:
                var = var_map[factor]
        if var is not None:
            gurobi_expr.addTerms(coeff, var)
        else:
            gurobi_expr.addConstant(coeff)
    elif sympy_expr in var_map:
        gurobi_expr.addTerms(1.0, var_map[sympy_expr])
    elif sympy_expr.is_number:
        gurobi_expr.addConstant(float(sympy_expr))

    return gurobi_expr

In [692]:
M = 20
EPS = 1
def add_constrs(variables, all_inds):
    for i, ind in enumerate(all_inds):
        rs = unify(ind)
        for r in rs:
            lhs = to_gp_linexpr(r.lhs)
            if r.rel_op == '>':
                model.addConstr(lhs >= EPS - M * (1 - variables[i]))
                model.addConstr(lhs <= M * variables[i])
            elif r.rel_op == '<=':
                model.addConstr(lhs <= M * (1 - variables[i]))
                # if ind = 0, at least one of the inequalities is violated
                if len(rs) == 1:
                    model.addConstr(lhs >= EPS - M * variables[i])
            else:
                raise ValueError

In [693]:
add_constrs(pos_inds, all_pos_inds)
add_constrs(neg_inds, all_neg_inds)

In [694]:
model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 23.4.0 23E224)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 155 rows, 69 columns and 520 nonzeros
Model fingerprint: 0xdc71f4b1
Variable types: 0 continuous, 69 integer (66 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [5e-03, 3e-02]
  Bounds range     [1e+00, 2e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 0.1805556
Presolve removed 8 rows and 4 columns
Presolve time: 0.01s
Presolved: 147 rows, 65 columns, 488 nonzeros
Variable types: 0 continuous, 65 integer (62 binary)

Root relaxation: objective 1.403356e-01, 7 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.14034    0    2    0.18056    0.14034  22.3%    

In [695]:
if model.status == GRB.OPTIMAL:
    print("Optimal solution found:")
    for v in model.getVars():
        print(f"{v.varName}: {v.x}, {not (not v.x)}")
    print(f"Objective value: {model.objVal}")
else:
    print("No feasible solution found.")

Optimal solution found:
pos_inds[0]: 0.0, False
pos_inds[1]: 1.0, True
pos_inds[2]: 0.0, False
pos_inds[3]: 1.0, True
pos_inds[4]: 1.0, True
pos_inds[5]: 0.0, False
pos_inds[6]: 0.0, False
pos_inds[7]: 0.0, False
pos_inds[8]: 0.0, False
pos_inds[9]: 1.0, True
pos_inds[10]: 1.0, True
pos_inds[11]: 0.0, False
pos_inds[12]: 1.0, True
pos_inds[13]: 0.0, False
pos_inds[14]: 0.0, False
pos_inds[15]: 0.0, False
pos_inds[16]: 0.0, False
pos_inds[17]: 0.0, False
pos_inds[18]: 0.0, False
pos_inds[19]: 1.0, True
pos_inds[20]: 1.0, True
pos_inds[21]: 1.0, True
pos_inds[22]: 1.0, True
pos_inds[23]: 0.0, False
pos_inds[24]: 0.0, False
pos_inds[25]: 0.0, False
pos_inds[26]: 1.0, True
pos_inds[27]: 1.0, True
pos_inds[28]: 1.0, True
pos_inds[29]: 1.0, True
pos_inds[30]: 0.0, False
pos_inds[31]: 0.0, False
pos_inds[32]: 0.0, False
pos_inds[33]: 0.0, False
pos_inds[34]: 0.0, False
pos_inds[35]: 0.0, False
neg_inds[0]: 1.0, True
neg_inds[1]: 1.0, True
neg_inds[2]: 1.0, True
neg_inds[3]: 1.0, True
neg_inds

In [696]:
# 3: 1,2,14
# 4: 0.5, 1.5, 4.5
def eval_f(n,a,b,c):
    tot = 0.
    vals = {x:a, y:b, z:c}
    for i in range(1,n+1):
        pos_c, pos_rs = H_XYZ[i]
        neg_c, neg_rs = H_YXZ[i]
        print(sum([sub(r, vals) for r in neg_rs]))
        tot += coef(i) * (pos_c - neg_c + sum([sub(r, vals) for r in pos_rs]) - sum([sub(r, vals) for r in neg_rs]))
    return tot - thresh(n)

In [697]:
eval_f(3,3,5,15)

0
4
10


0.04351851851851851

In [682]:
H_XYZ[2]

(1, [[3*x <= y], [2*x <= z], [2*x <= y], [3*x <= z], [x <= -y + z]])